In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

#openai_api_key가져오기
from openai import OpenAI  # 주석처리
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기
#openai_api_key가져오기

llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈습니다! 당신은 어떻게 지내고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 12, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bx72SriKftfKpiSntZm30AviJcnMQ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ec8462a1-0704-4c77-942c-fa684d255214-0', usage_metadata={'input_tokens': 12, 'output_tokens': 18, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time


In [4]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [5]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qDnE3IhkowQxzKL8NWplyOlX', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bx72TrO0DJ6FbQYoH5YZislOQkETN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--57137d77-7f5f-4604-86ff-4a308cdd0854-0', tool_ca

In [6]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan) 현재시각 2025-07-25 16:21:55 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qDnE3IhkowQxzKL8NWplyOlX', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bx72TrO0DJ6FbQYoH5YZislOQkETN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--57137d77-7f5f-4604-86ff-4a308cdd0854-0', tool_

In [7]:
llm_with_tools.invoke(messages)

AIMessage(content='부산은 지금 2025년 7월 25일 오후 4시 21분 55초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 192, 'total_tokens': 219, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bx72UiQoDCBHm4LADqsFUDq84Ia1S', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f800b1e2-4c9e-4e92-b6c4-e2e86cad6d70-0', usage_metadata={'input_tokens': 192, 'output_tokens': 27, 'total_tokens': 219, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")


In [9]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)

In [10]:
messages.append(HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_rkBBmeF7qfyXFWr0gjfWgL8O', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 283, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bx72WzYztB1OOUWogFa6QUifMpGlx', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--8d14f1a9-da89-4d43-984a-89289b98b2d9-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_rkBBmeF7qfyXFWr0gjfWgL8O', 'type': 'tool_call'}] usage_metadata={'inp

In [11]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-06-25 00:00:00-04:00 | 342.7  | 343    | 320.4  |  327.55 | 1.19845e+08 |           0 |              0 |\n| 2025-06-26 00:00:00-04:00 | 324.61 | 331.05 | 323.61 |  325.78 | 8.04409e+07 |           0 |              0 |\n| 2025-06-27 00:00:00-04:00 | 324.51 | 329.34 | 317.5  |  323.63 | 8.9067e+07  |           0 |              0 |\n| 2025-06-30 00:00:00-04:00 | 319.9  | 325.58 | 316.6  |  317.66 | 7.66951e+07 |           0 |              0 |\n| 2025-07-01 00:00:00-04:00 | 298.46 | 305.89 | 293.21 |  300.71 | 1.45086e+08 |           0 |              0 |\n| 2025-07-02 00:00:00-04:00 | 312.63 | 316.83 | 303.82 |  315.65 | 1.19484e+08 |           0 |              0 |\n| 2025-07-03 00:00:00-04:0

In [12]:
llm_with_tools.invoke(messages)

AIMessage(content='한 달 전인 2025년 6월 25일의 테슬라(TSLA) 주가 폐쇄가는 327.55달러였고, 현재 주가(2025년 7월 24일)는 305.30달러입니다. \n\n따라서, 테슬라의 주가는 한 달 전에 비해 내렸습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 1586, 'total_tokens': 1665, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bx72ZilME0sJspOgz9JMpvvLyibKe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--af1c91aa-02aa-482f-a651-d7432d5d427b-0', usage_metadata={'input_tokens': 1586, 'output_tokens': 79, 'total_tokens': 1665, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 08-4 스트림 출력

In [13]:
for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점에 대해 이야기해줘.")]):
    print(c.content, end='|') 

|안|녕하세요|!| 한국| 사회|의| 문제|점|에| 대해| 이야기|해|볼|게|요|.| 한국| 사회|는| 여러| 가지| 복|잡|한| 문제|를| 안|고| 있습니다|.| 몇| 가지| 주요|한| 문제|점|은| 다음|과| 같습니다|.

|1|.| **|고|용| 불|안|정|**|:| 청|년|층|의| 실|업|률|이| 높|고|,| 비|정|규|직| 일|자|리가| 많|아| 경제|적| 불|안|정|성이| 커|지고| 있습니다|.| 이는| 많은| 젊|은|이|들이| 안정|적인| 직|장을| 찾|기| 어려|운| 상황|을| 초|래|하고| 있습니다|.

|2|.| **|양|극|화|**|:| 경제|적|,| 사회|적| 양|극|화|가| 심|화|되고| 있습니다|.| 부|유|층|과| 저|소|득|층| 간|의| 격|차|가| 커|지고|,| 이는| 사회|적| 갈|등|을| 유|발|하는| 주요| 원|인| 중| 하나|입니다|.

|3|.| **|주|거| 문제|**|:| 큰| 도시|,| 특히| 서울|의| 주|거| 비용|이| 급|증|하면서| 많은| 사람들이| 주|택| 문제|로| 어려|움을| 겪|고| 있습니다|.| 이|로| 인해| 젊|은| 세|대|가| 독|립|적으로| 생활|하기| 어려|워|지고| 있습니다|.

|4|.| **|정|신| 건강| 문제|**|:| 정신| 건강| 문제가| 사회|적으로| 큰| 이|슈|로| 떠|오|르고| 있습니다|.| 특히|,| 경쟁|이| 치|열|한| 사회|에서| 스트|레스|와| 우|울|증|을| 겪|는| 사람들이| 많|지만|,| 여|전히| 이에| 대한| 인|식|이| 부족|하고| 지원| 체|계|도| 미|비|한| 상황|입니다|.

|5|.| **|고|령|화| 사회|**|:| 한국|은| 세계|에서| 가장| 빠|르게| 고|령|화|가| 진행|되고| 있는| 나라| 중| 하나|입니다|.| 이|로| 인해| 노|인| 빈|곤| 문제|와| 세|대| 간| 갈|등|이| 심|화|되고| 있습니다|.

|6|.| **|성|차|별|과| 성|폭|력|**|:| 성| 평|등| 문제가| 여|전히| 심|각|하고|,| 성|폭|력| 사건|과

In [14]:
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

response = llm_with_tools.stream(messages)

# 파편화된 tool_call 청크를 하나로 합치기 
is_first = True
for chunk in response:    
    print("chunk type: ", type(chunk))
    
    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk
    
    print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'type': 'tool_c

In [15]:
gathered

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'service_tier': 'default'}, id='run--a1456b4a-80d2-4063-a129-393970910753', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"Busan"}', 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'index': 0, 'type': 'tool_call_chunk'}])

In [16]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # tool_dict를 사용하여 도구 이름으로 도구 함수를 선택
    print(tool_call["args"]) # 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan) 현재시각 2025-07-25 16:22:09 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"Busan"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'service_tier': 'default'}, id='run--a1456b4a-80d2-4063-a129-393970910753', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"Busan"}', 'id': 'call_od2VseNc9XZwRn8dnKtdDtiT', 'index': 0, 'type': 'tool_call_chunk'}]),
 ToolMessage(conten

In [17]:
for c in llm_with_tools.stream(messages):
    print(c.content, end='|')

|부|산|은| 지금| |202|5|년| |7|월| |25|일| |16|시| |22|분|입니다|.||